In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import validation_curve
from sklearn.metrics import make_scorer
from sklearn import metrics as mt
from sklearn.metrics import f1_score as f1
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import os

## Import dataset

In [21]:
#training files
df_etd_training = pd.read_csv("../Data/chat_pattern/training/etd_pattern_training.csv")
df_ps_training = pd.read_csv("../Data/chat_pattern/training/ps_pattern_training.csv")
df_ngram_training = pd.read_csv("../Data/chat_pattern/training/ngram_training.csv")
df_pos_training = pd.read_csv("../Data/chat_pattern/training/pos_training.csv")

#testing files
df_etd_testing = pd.read_csv("../Data/chat_pattern/testing/etd_pattern_testing.csv")
df_ps_testing = pd.read_csv("../Data/chat_pattern/testing/ps_pattern_testing.csv")
df_ngram_testing = pd.read_csv("../Data/chat_pattern/testing/ngram_testing.csv")
df_pos_testing = pd.read_csv("../Data/chat_pattern/testing/pos_testing.csv")

#label
df_label_training = pd.read_csv("../Data/chat_pattern/chat_annotation_1000_pos.csv")
df_label_testing = pd.read_csv("../Data/chat_pattern/chat_testing_200.csv")

In [ ]:
X_etd_train = df_etd_training.values
X_ps_train = df_ps_training.values
X_ngram_train = df_ngram_training.values
X_pos_train = df_pos_training.values

X_etd_ngram_train = np.hstack((X_etd_train, X_ngram_train))
X_ps_ngram_train = np.hstack((X_ps_train, X_ngram_train))
X_etd_pos_train = np.hstack((X_etd_train, X_pos_train))
X_ps_pos_train = np.hstack((X_ps_train, X_pos_train))
X_ngram_pos_train = np.hstack((X_ngram_train, X_pos_train))

X_all_etd_train = np.hstack((X_etd_ngram_train, X_pos_train))
X_all_ps_train = np.hstack((X_ps_ngram_train, X_pos_train))

In [83]:
X_etd_test = df_etd_testing.values
X_ps_test = df_ps_testing.values
X_ngram_test = df_ngram_testing.values
X_pos_test = df_pos_testing.values


X_etd_ngram_test = np.hstack((X_etd_test, X_ngram_test))
X_etd_pos_test = np.hstack((X_etd_test, X_pos_test))
X_ngram_pos_test = np.hstack((X_ngram_test, X_pos_test))
X_ps_ngram_test = np.hstack((X_ps_test, X_ngram_test))
X_ps_pos_test = np.hstack((X_ps_test, X_pos_test))

X_all_etd_test = np.hstack((X_etd_ngram_test, X_pos_test))
X_all_ps_test = np.hstack((X_ngram_pos_test, X_pos_test))

In [65]:
y_etd_train = df_label_training["y_ETD"].values
y_ps_train = df_label_training["y_PS"].values
y_etd_test = df_label_testing["y_ETD"].values
y_ps_test = df_label_testing["y_PS"].values

## Train Test Split (10-fold)

In [4]:
df_all = pd.read_csv("../Data/chat_pattern/chat_test_1000.csv")
df_all.head()

,source,issue,PS_sent,predicted_PS,ETD_sent,predicted_ETD,y_PS,y'_PS,Acc_PS,y_ETD,y'_ETD,Acc_ETD,openAI_PS,Inferred_PS_pattern,y''_PS,ACC_AI_PS,openAI_ETD,Inferred_ETD_pattern,y''_ETD,ACC_AI_ETD
0,angular,"Hi, when I am trying to apply class dynamicall...",T2: it is messing up my material css i.e I am ...,"PS_NEG_VERB,PS_NEG_AUX_ADV_ADJ,PS_NEG_AUX_VERB,",T1: when I am trying to apply class dynamicall...,"ETD_TRYING_TO,",1,1,1,1,1,1,"The sentence """""" it is messing up my material ...",PS_NEG_VERB,1.0,1.0,"The sentence """""" when I am trying to apply cla...",NaN,1.0,1.0
1,angular,is it possible to get a RouteConfig matched ag...,NaN,NaN,T1: I'm trying to create a Breadcrumb componen...,"ETD_BE_POSSIBLE_TO,ETD_TRYING_TO,ETD_WOULD_LIKE,",0,0,1,1,1,1,No patterns found. The answer is NO.,NaN,0.0,1.0,"The sentence """""" I'm trying to create a Breadc...",NaN,1.0,1.0
2,NaN,I have angular running from a .Net Core server...,T1: The server won't even load the app (by des...,"PS_NEG_AUX_VERB,",NaN,NaN,1,1,1,0,0,1,No patterns found. The answer is NO.,NaN,0.0,0.0,"The sentence """""" What's the best way for the a...",new,1.0,0.0
3,angular,"Hello everyone, I want to do sub menu with sea...",T2: but i have problem in sub menu,"PS_PROBLEM,",T1: I want to do sub menu with searh like in g...,"ETD_WANT_TO,ETD_CAN_QUESTION,",1,1,1,1,1,1,"The sentence """""" i have problem in sub menu wi...","""problem"" is the [error-term]",1.0,1.0,"The sentence ""i have problem in sub menu with ...",new,1.0,1.0
4,angular,I'm struggling with getting this logic\n ...,T1: I'm struggling with getting this logic,"PS_STRUGGLING,PS_VERB_NO,",T2: I wanted to set a default value to the dro...,"ETD_WANT_TO,",1,1,1,1,1,1,"The sentence ""I'm struggling with getting this...",NaN,1.0,1.0,"The sentence """"""I wanted to set a default valu...",NaN,1.0,1.0


In [27]:
df_random = df_all.sample(n=len(df_all),ignore_index=True)
df_split = np.array_split(df_random, 10)

In [29]:
import os

train_path = "../Data/chat_pattern/cross_validate/"
for i in range(len(df_split)):
    df_split = np.array_split(df_random, 10)
    df_test = df_split[i]
    df_val = df_split[len(df_split)-i-1]
    frame_train = [df_split[index] for index in range(len(df_split)) if index != i and index != (len(df_split)-i-1)]
    df_train = pd.concat(frame_train)
    
    if not os.path.exists(train_path+str(i)+"/"):
        os.makedirs(train_path+str(i)+"/")
        
    df_train.to_csv(train_path+str(i)+"/"+"train_800.csv",index = None)
    df_val.to_csv(train_path+str(i)+"/"+"val_100.csv",index = None)
    df_test.to_csv(train_path+str(i)+"/"+"test_100.csv",index = None)

In [3]:
X_etd_train_list = []
X_ps_train_list = []
X_ngram_train_list = []
X_pos_train_list = []
X_etd_ngram_train_list = []
X_ps_ngram_train_list = []
X_etd_pos_train_list = []
X_ps_pos_train_list = []
X_ngram_pos_train_list = []
X_all_etd_train_list = []
X_all_ps_train_list = []

X_etd_test_list = []
X_ps_test_list = []
X_ngram_test_list = []
X_pos_test_list = []
X_etd_ngram_test_list = []
X_ps_ngram_test_list = []
X_etd_pos_test_list = []
X_ps_pos_test_list = []
X_ngram_pos_test_list = []
X_all_etd_test_list = []
X_all_ps_test_list = []

X_etd_val_list = []
X_ps_val_list = []
X_ngram_val_list = []
X_pos_val_list = []
X_etd_ngram_val_list = []
X_ps_ngram_val_list = []
X_etd_pos_val_list = []
X_ps_pos_val_list = []
X_ngram_pos_val_list = []
X_all_etd_val_list = []
X_all_ps_val_list = []

y_etd_train_list = []
y_ps_train_list = []
y_etd_test_list = []
y_ps_test_list = []
y_etd_val_list = []
y_ps_val_list = []

In [4]:
def prepare_10fold_data(path):
    dir_list = os.listdir(path)
    for folder in dir_list:
        #training files
        df_etd_train = pd.read_csv(path+folder+"/train/etd_pattern_train.csv")
        df_ps_train = pd.read_csv(path+folder+"/train/ps_pattern_train.csv")
        df_ngram_train = pd.read_csv(path+folder+"/train/ngram_train.csv")
        df_pos_train = pd.read_csv(path+folder+"/train/pos_train.csv")

        #test files
        df_etd_test = pd.read_csv(path+folder+"/test/etd_pattern_test.csv")
        df_ps_test = pd.read_csv(path+folder+"/test/ps_pattern_test.csv")
        df_ngram_test = pd.read_csv(path+folder+"/test/ngram_test.csv")
        df_pos_test = pd.read_csv(path+folder+"/test/pos_test.csv")
        
        #validation files
        df_etd_val = pd.read_csv(path+folder+"/val/etd_pattern_val.csv")
        df_ps_val = pd.read_csv(path+folder+"/val/ps_pattern_val.csv")
        df_ngram_val = pd.read_csv(path+folder+"/val/ngram_val.csv")
        df_pos_val = pd.read_csv(path+folder+"/val/pos_val.csv")

        #label
        df_label_train = pd.read_csv(path+folder+"/train_800.csv")
        df_label_test = pd.read_csv(path+folder+"/test_100.csv")
        df_label_val = pd.read_csv(path+folder+"/val_100.csv")
        
        #train
        X_etd_train = df_etd_train.values
        X_ps_train = df_ps_train.values
        X_ngram_train = df_ngram_train.values
        X_pos_train = df_pos_train.values

        X_etd_ngram_train = np.hstack((X_etd_train, X_ngram_train))
        X_ps_ngram_train = np.hstack((X_ps_train, X_ngram_train))
        X_etd_pos_train = np.hstack((X_etd_train, X_pos_train))
        X_ps_pos_train = np.hstack((X_ps_train, X_pos_train))
        X_ngram_pos_train = np.hstack((X_ngram_train, X_pos_train))

        X_all_etd_train = np.hstack((X_etd_ngram_train, X_pos_train))
        X_all_ps_train = np.hstack((X_ps_ngram_train, X_pos_train))
        
        #test
        X_etd_test = df_etd_test.values
        X_ps_test = df_ps_test.values
        X_ngram_test = df_ngram_test.values
        X_pos_test = df_pos_test.values

        X_etd_ngram_test = np.hstack((X_etd_test, X_ngram_test))
        X_etd_pos_test = np.hstack((X_etd_test, X_pos_test))
        X_ngram_pos_test = np.hstack((X_ngram_test, X_pos_test))
        X_ps_ngram_test = np.hstack((X_ps_test, X_ngram_test))
        X_ps_pos_test = np.hstack((X_ps_test, X_pos_test))

        X_all_etd_test = np.hstack((X_etd_ngram_test, X_pos_test))
        X_all_ps_test = np.hstack((X_ps_ngram_test, X_pos_test))
        
        #val
        X_etd_val = df_etd_val.values
        X_ps_val = df_ps_val.values
        X_ngram_val = df_ngram_val.values
        X_pos_val = df_pos_val.values

        X_etd_ngram_val = np.hstack((X_etd_val, X_ngram_val))
        X_etd_pos_val = np.hstack((X_etd_val, X_pos_val))
        X_ngram_pos_val = np.hstack((X_ngram_val, X_pos_val))
        X_ps_ngram_val = np.hstack((X_ps_val, X_ngram_val))
        X_ps_pos_val = np.hstack((X_ps_val, X_pos_val))

        X_all_etd_val = np.hstack((X_etd_ngram_val, X_pos_val))
        X_all_ps_val = np.hstack((X_ps_ngram_val, X_pos_val))
        
        y_etd_train = df_label_train["y_ETD"].values
        y_ps_train = df_label_train["y_PS"].values
        y_etd_test = df_label_test["y_ETD"].values
        y_ps_test = df_label_test["y_PS"].values
        y_etd_val = df_label_val["y_ETD"].values
        y_ps_val = df_label_val["y_PS"].values
        
        X_etd_train_list.append(X_etd_train)
        X_ps_train_list.append(X_ps_train)
        X_ngram_train_list.append(X_ngram_train)
        X_pos_train_list.append(X_pos_train)
        X_etd_ngram_train_list.append(X_etd_ngram_train)
        X_ps_ngram_train_list.append(X_ps_ngram_train)
        X_etd_pos_train_list.append(X_etd_pos_train)
        X_ps_pos_train_list.append(X_ps_pos_train)
        X_ngram_pos_train_list.append(X_ngram_pos_train)
        X_all_etd_train_list.append(X_all_etd_train)
        X_all_ps_train_list.append(X_all_ps_train)
        
        X_etd_test_list.append(X_etd_test)
        X_ps_test_list.append(X_ps_test)
        X_ngram_test_list.append(X_ngram_test)
        X_pos_test_list.append(X_pos_test)
        X_etd_ngram_test_list.append(X_etd_ngram_test)
        X_ps_ngram_test_list.append(X_ps_ngram_test)
        X_etd_pos_test_list.append(X_etd_pos_test)
        X_ps_pos_test_list.append(X_ps_pos_test)
        X_ngram_pos_test_list.append(X_ngram_pos_test)
        X_all_etd_test_list.append(X_all_etd_test)
        X_all_ps_test_list.append(X_all_ps_test)
        
        X_etd_val_list.append(X_etd_val)
        X_ps_val_list.append(X_ps_val)
        X_ngram_val_list.append(X_ngram_val)
        X_pos_val_list.append(X_pos_val)
        X_etd_ngram_val_list.append(X_etd_ngram_val)
        X_ps_ngram_val_list.append(X_ps_ngram_val)
        X_etd_pos_val_list.append(X_etd_pos_val)
        X_ps_pos_val_list.append(X_ps_pos_val)
        X_ngram_pos_val_list.append(X_ngram_pos_val)
        X_all_etd_val_list.append(X_all_etd_val)
        X_all_ps_val_list.append(X_all_ps_val)
        
        y_etd_train_list.append(y_etd_train)
        y_ps_train_list.append(y_ps_train)
        y_etd_test_list.append(y_etd_test)
        y_ps_test_list.append(y_ps_test)
        y_etd_val_list.append(y_etd_val)
        y_ps_val_list.append(y_ps_val)


In [5]:
prepare_10fold_data("../Data/chat_pattern/cross_validate/")

## Model

In [131]:
def train_evaluate(X_train, y_train, X_test, y_test):
    y_test_model = []
    yhat_model = []
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    
    svc = SVC()
    grid_search = GridSearchCV(svc, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    print('CV Train score: {:.2f}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
    predictions = grid_search.predict(X_test)
    precison = mt.precision_score(y_test, predictions)
    recall = mt.recall_score(y_test, predictions)
    score = mt.f1_score(y_test, predictions)
    
    print("precision:",round(precison,3),"recall:",round(recall,3),"F1:",round(score,3))
    
    for val in zip(y_test, predictions):
        yhat_model.append(val[1])
        y_test_model.append(val[0])
    
    return precison,recall,score, y_test_model, yhat_model

In [ ]:
def train_evaluate_2(X_train, y_train, X_test, y_test):
    y_test_model = []
    yhat_model = []
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    
    svc = SVC()
    scoring = {'Precision': 'precision', 'Recall':'recall', 'F1': 'f1'}
    grid_search = GridSearchCV(svc, param_grid, cv=10, scoring = scoring, refit = "F1")
    
    #X_train = np.vstack((X_train, X_test))
    #y_train = np.concatenate((y_train, y_test))
    
    grid_search.fit(X_train, y_train)
    
    #print('CV Train score: {:.2f}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
    results = grid_search.cv_results_
    
    precison = np.average(results["mean_test_Precision"])
    recall = np.average(results["mean_test_Recall"])
    score = np.average(results["mean_test_F1"])
    
    print("precision:",round(precison,3),"recall:",round(recall,3),"F1:",round(score,3))
    
    return results

In [6]:
def cross_validate(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
    
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    precision_list = []
    recall_list = []
    f1_list = []
    
    for X_train,y_train,X_test,y_test,X_val,y_val in zip(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
        svc = SVC()
        grid_search = GridSearchCV(svc, param_grid, cv=5)
        grid_search.fit(X_val, y_val)
    
        #print('CV Train score: {:.2f}'.format(grid_search.best_score_))
        #print('Best parameters: {}'.format(grid_search.best_params_))
        
        clf = SVC(**grid_search.best_params_)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        precison = mt.precision_score(y_test, predictions)
        recall = mt.recall_score(y_test, predictions)
        score = mt.f1_score(y_test, predictions)
        
        precision_list.append(precison)
        recall_list.append(recall)
        f1_list.append(score)
    
    print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [10]:
def cross_validate1(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
    
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    
    for X_train,y_train,X_test,y_test,X_val,y_val in zip(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
        precision_list = []
        recall_list = []
        f1_list = []
        
        svc = SVC()
        grid_search = GridSearchCV(svc, param_grid, cv=5)
        grid_search.fit(X_val, y_val)
    
        #print('CV Train score: {:.2f}'.format(grid_search.best_score_))
        #print('Best parameters: {}'.format(grid_search.best_params_))
        
        clf = SVC(**grid_search.best_params_)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        precison = mt.precision_score(y_test, predictions)
        recall = mt.recall_score(y_test, predictions)
        score = mt.f1_score(y_test, predictions)
        
        precision_list.append(precison)
        recall_list.append(recall)
        f1_list.append(score)
    
    print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

## Training

In [7]:
# 1. ETD: pattern
cross_validate(X_etd_train_list, y_etd_train_list, X_etd_test_list, y_etd_test_list, X_etd_val_list, y_etd_val_list)

precision: 0.877 recall: 0.778 F1: 0.824


In [8]:
# 2. ETD: pos
cross_validate(X_pos_train_list, y_etd_train_list, X_pos_test_list, y_etd_test_list, X_pos_val_list, y_etd_val_list)

precision: 0.692 recall: 0.793 F1: 0.733


In [14]:
# 3. ETD: n-gram
cross_validate(X_ngram_train_list, y_etd_train_list, X_ngram_test_list, y_etd_test_list,X_ngram_val_list, y_etd_val_list)


precision: 0.875 recall: 0.656 F1: 0.75


In [17]:
# 4. ETD: pattern + pos
cross_validate(X_etd_pos_train_list, y_etd_train_list, X_etd_pos_test_list, y_etd_test_list,X_etd_pos_val_list, y_etd_val_list)


precision: 0.73 recall: 0.844 F1: 0.783


In [18]:
# 5. ETD: pattern + n-gram
cross_validate(X_etd_ngram_train_list, y_etd_train_list, X_etd_ngram_test_list, y_etd_test_list, X_etd_ngram_val_list, y_etd_val_list)


precision: 0.895 recall: 0.797 F1: 0.843


In [19]:
# 6. ETD: n-gram + pos
cross_validate(X_ngram_pos_train_list, y_etd_train_list, X_ngram_pos_test_list, y_etd_test_list,X_ngram_pos_val_list, y_etd_val_list)


precision: 0.806 recall: 0.781 F1: 0.794


In [20]:
# 7. ETD: pattern + n-gram + pos
cross_validate(X_all_etd_train_list, y_etd_train_list, X_all_etd_test_list, y_etd_test_list,X_all_etd_val_list, y_etd_val_list)


precision: 0.836 recall: 0.797 F1: 0.816


In [9]:
# 8. PS: pattern
cross_validate(X_ps_train_list, y_ps_train_list, X_ps_test_list, y_ps_test_list, X_ps_val_list, y_ps_val_list)

precision: 0.872 recall: 0.889 F1: 0.88


In [10]:
# 9. PS: pos
cross_validate(X_pos_train_list, y_ps_train_list, X_pos_test_list, y_ps_test_list,X_pos_val_list, y_ps_val_list)

precision: 0.752 recall: 0.907 F1: 0.815


In [22]:
# 10. PS: n-gram
cross_validate(X_ngram_train_list, y_ps_train_list, X_ngram_test_list, y_ps_test_list,X_ngram_val_list, y_ps_val_list)

precision: 0.959 recall: 0.635 F1: 0.764


In [23]:
# 11. PS: pattern + pos
cross_validate(X_ps_pos_train_list, y_ps_train_list, X_ps_pos_test_list, y_ps_test_list,X_ps_pos_val_list, y_ps_val_list)


precision: 0.836 recall: 0.824 F1: 0.83


In [24]:
# 12. PS: pattern + n-gram
cross_validate(X_ps_ngram_train_list, y_ps_train_list, X_ps_ngram_test_list, y_ps_test_list,X_ps_ngram_val_list, y_ps_val_list)


precision: 0.93 recall: 0.716 F1: 0.809


In [25]:
# 13. PS: n-gram + pos
cross_validate(X_ngram_pos_train_list, y_ps_train_list, X_ngram_pos_test_list, y_ps_test_list,X_ngram_pos_val_list, y_ps_val_list)


precision: 0.845 recall: 0.811 F1: 0.828


In [30]:
# 14. PS: pattern + n-gram + pos
cross_validate(X_all_ps_train_list, y_ps_train_list, X_all_ps_test_list, y_ps_test_list,X_all_ps_val_list, y_ps_val_list)


precision: 0.861 recall: 0.838 F1: 0.849
